In [1]:
import os
txt_files = [file for file in os.listdir() if file.endswith('.txt')]

txt_files.sort()

combined_text = ""

for book in txt_files:
  with open(book,'r',encoding='utf-8') as file:
    combined_text += file.read() + "\n\n"

print(combined_text[:1000])



M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.

Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere.

The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn’t think they could bear it if anyone found out about the Potters. Mrs. Potter was Mrs. Dursley’s sister, but they hadn’t met for several ye

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import re

In [3]:
tokenizer = Tokenizer()

In [4]:
tokenizer.fit_on_texts([combined_text])

In [5]:
len(tokenizer.word_index)

10441

In [6]:
input_sequences = []
for line in combined_text.split('.'):  # Assuming sentences are split by periods
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [7]:
max_len =  max([len(x) for x in input_sequences])

In [8]:
max_len

213

In [9]:
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

In [10]:
X = padded_input_sequences[:,:-1]
y = padded_input_sequences[:,-1]

In [11]:
total_words = len(tokenizer.word_index) + 1

In [12]:
from scipy.sparse import csr_matrix
y_sparse = csr_matrix((np.ones(len(y)), (np.arange(len(y)), y)), shape=(len(y), total_words))

In [13]:
def data_generator(X, y_sparse, batch_size, total_words):
    while True:
        for start in range(0, len(X), batch_size):
            end = min(start + batch_size, len(X))
            y_batch = y_sparse[start:end].toarray()
            yield X[start:end], y_batch

batch_size = 128
steps_per_epoch = len(X) // batch_size

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_len-1))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


history = model.fit(data_generator(X, y_sparse, batch_size, total_words), epochs=100, steps_per_epoch=steps_per_epoch)


Epoch 1/100
1243/1243 [==============================] - 61s 44ms/step - loss: 6.6494 - accuracy: 0.0669
Epoch 2/100
1243/1243 [==============================] - 44s 35ms/step - loss: 5.8219 - accuracy: 0.1189
Epoch 3/100
1243/1243 [==============================] - 45s 36ms/step - loss: 5.3856 - accuracy: 0.1409
Epoch 4/100
1243/1243 [==============================] - 44s 36ms/step - loss: 5.0884 - accuracy: 0.1563
Epoch 5/100
1243/1243 [==============================] - 44s 36ms/step - loss: 4.8429 - accuracy: 0.1678
Epoch 6/100
1243/1243 [==============================] - 45s 36ms/step - loss: 4.6665 - accuracy: 0.1778
Epoch 7/100
1243/1243 [==============================] - 45s 36ms/step - loss: 4.4889 - accuracy: 0.1886
Epoch 8/100
1243/1243 [==============================] - 45s 36ms/step - loss: 4.3061 - accuracy: 0.2016
Epoch 9/100
1243/1243 [==============================] - 44s 36ms/step - loss: 4.1316 - accuracy: 0.2160
Epoch 10/100
1243/1243 [==============================]

In [22]:
import numpy as np

def generate_text(seed_text, model, tokenizer, max_sequence_len, num_words_to_generate):
    for _ in range(num_words_to_generate):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs, axis=-1)[0]
        predicted_word = tokenizer.index_word[predicted_index]

        seed_text += " " + predicted_word
    return seed_text


seed_text = "who is dumbledore? "
num_words_to_generate = 50
generated_text = generate_text(seed_text, model, tokenizer, max_len, num_words_to_generate)

print(generated_text)


who is dumbledore?  the most wizard of hogwarts and the sorting hat is match in the world ” said harry loudly to the dark one of the house and the note was still around the castle with the second harry had never seen before it was downstairs to the fire which was a
